# Simple place cell statistics for open-field, light and dark trials

The spatial maps are calculated and saved in `open_field_light_dark_maps.ipynb`. We won't to this here.

We are getting the data in this notebook. The figures will be done in the next notebook.

In [1]:
%load_ext autoreload
%autoreload 2
%run ../setup_project.py
%run ../neuronAutopi.py

prepareSessionsForSpatialAnalysisProject(sSesList,myProject.sessionList)

Project name: autopi_ca1
dataPath: /ext_drives/d80/Jazi_etal_2023/autopi_ca1
Reading /ext_drives/d80/Jazi_etal_2023/autopi_ca1/sessionList
We have 39 testing sessions in the list
See myProject and sSesList objects
Loading Animal_pose and Spike_train, sSes.ap and sSes.cg


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:09<00:00,  4.12it/s]

Loading ses.trial_table_simple as ses.trials
Create condition intervals in ses.intervalDict


In [2]:
for ses, sSes in tqdm(zip(myProject.sessionList,sSesList)):
    getSearchHomingIntervals(ses,sSes)

39it [00:00, 48.52it/s]


## Mean firing rate and information score per condition

We will get mean firing rates and info scores per cell per condition. We return a Pandas DataFrame for each session.

In [3]:
def getMapStats(ses,sSes,interName="circ80",onlyArena=False,conditionName="x"):
    """
    Calculate the mean firing rate of the neuron
    
    Arguments: 
    ses: autopipy session
    sSes: spikeA session
    interName: name of the interval to analyze
    onlyArena: remove spatial data points outside the arena
    conditionName: condition name for the data frame
        
    Return:
    Pandas DataFrame with cellId,condition,meanRate columns
    """ 
    
    myDict={}
    xy_range=np.array([[-50,-90],[50,60]])
    
    sSes.ap.set_intervals(ses.intervalDict[interName])
    
    if onlyArena:
        # this should come after setting the time intervals, will be reset when calling sSes.ap.set_intervals again
        sSes.ap.invalid_outside_spatial_area(environment_shape= "circle",radius=43.0,center=np.array([0,0])) ## remove the bridge and homebase
    
    # set the intervals for each neuron
    for n in sSes.cg.neuron_list:
        n.spike_train.set_intervals(ses.intervalDict[interName])
        
        n.spatial_properties.firing_rate_map_2d(cm_per_bin =3, smoothing_sigma_cm = 5, smoothing=True,xy_range=xy_range)
        
    myDict["peakRate"] = [ np.nanmax(n.spatial_properties.firing_rate_map) for n in sSes.cg.neuron_list] # not affected by onlyArena!!!!
    myDict["meanRate"] = [ n.spike_train.mean_firing_rate() for n in sSes.cg.neuron_list]
    
    for n in sSes.cg.neuron_list:
        n.spike_train.set_intervals(ses.intervalDict[interName])
        n.spatial_properties.firing_rate_map_2d(cm_per_bin = 3, smoothing=False,xy_range = xy_range) # no smoothing when calculating these maps

    myDict["info"] = [ n.spatial_properties.information_score() for n in sSes.cg.neuron_list]
    myDict["sparsity"] = [ n.spatial_properties.sparsity_score() for n in sSes.cg.neuron_list]
    myDict["condition"] = conditionName    
    myDict["session"] = sSes.name
    myDict["mouse"] = sSes.subject
    myDict["cellId"] = [sSes.name+"_"+n.name for n in sSes.cg.neuron_list]
        
    #create a DataFrame from the dictionary
    res = pd.DataFrame(myDict)
    return res


### One session

In [4]:
ses,sSes = list(zip(myProject.sessionList,sSesList))[0]
res=getMapStats(ses,sSes,interName="circ80",onlyArena=False,conditionName="circ80")


/home/maryam/repo/spikeA/spikeA/Spatial_properties.py:660: RuntimeWarning: invalid value encountered in double_scalars
  return 1-(((np.nansum(p*v))**2)/np.nansum(p*(v**2)))


### All sessions


In [5]:
conditions = ["circ80","circ80_1","circ80_2","task","task_1","task_2","light","light","dark","dark","light_1","light_2","dark_1","dark_2","all_light","all_light_1","all_light_2","all_dark","all_dark_1","all_dark_2"]
onlyArena = [False,False,False,False,False,False,False,True,False,True,True,True,True,True,True,True,True,True,True,True]
keys = ["circ80","circ80_1","circ80_2","task","task_1","task_2","light","light_arena","dark","dark_arena","light_1","light_2","dark_1","dark_2","all_light","all_light_1","all_light_2","all_dark","all_dark_1","all_dark_2"]
print(list(zip(keys,conditions,onlyArena)))

[('circ80', 'circ80', False), ('circ80_1', 'circ80_1', False), ('circ80_2', 'circ80_2', False), ('task', 'task', False), ('task_1', 'task_1', False), ('task_2', 'task_2', False), ('light', 'light', False), ('light_arena', 'light', True), ('dark', 'dark', False), ('dark_arena', 'dark', True), ('light_1', 'light_1', True), ('light_2', 'light_2', True), ('dark_1', 'dark_1', True), ('dark_2', 'dark_2', True), ('all_light', 'all_light', True), ('all_light_1', 'all_light_1', True), ('all_light_2', 'all_light_2', True), ('all_dark', 'all_dark', True), ('all_dark_1', 'all_dark_1', True), ('all_dark_2', 'all_dark_2', True)]


In [6]:
%%time
dfMapStats=pd.DataFrame()
for key,cond,onlyA in zip(keys,conditions,onlyArena):
    print(key,cond,onlyA)
    condDf = pd.concat([getMapStats(ses,sSes,cond,onlyA,key) for ses,sSes in zip(myProject.sessionList,sSesList)])
    dfMapStats = pd.concat([dfMapStats,condDf])

circ80 circ80 False
circ80_1 circ80_1 False


/home/maryam/repo/spikeA/spikeA/Spatial_properties.py:660: RuntimeWarning: invalid value encountered in double_scalars
  return 1-(((np.nansum(p*v))**2)/np.nansum(p*(v**2)))


circ80_2 circ80_2 False


/home/maryam/repo/spikeA/spikeA/Spatial_properties.py:660: RuntimeWarning: invalid value encountered in double_scalars
  return 1-(((np.nansum(p*v))**2)/np.nansum(p*(v**2)))


task task False


...

all_dark_1 all_dark_1 True


/home/maryam/repo/spikeA/spikeA/Spatial_properties.py:660: RuntimeWarning: invalid value encountered in double_scalars
  return 1-(((np.nansum(p*v))**2)/np.nansum(p*(v**2)))


all_dark_2 all_dark_2 True


/home/maryam/repo/spikeA/spikeA/Spatial_properties.py:660: RuntimeWarning: invalid value encountered in double_scalars
  return 1-(((np.nansum(p*v))**2)/np.nansum(p*(v**2)))


CPU times: user 13min 56s, sys: 1min 1s, total: 14min 57s
Wall time: 14min 57s


In [7]:
fn=myProject.dataPath+"/results/myMapStats.csv"
print(fn)
dfMapStats.to_csv(fn,index=False)

/ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/results/myMapStats.csv


In [8]:
print('we are here!')

we are here!


# Map similarity for a single neuron across conditions

We can calculate the correlations between firing rate maps.


In [9]:
from scipy.stats import pearsonr
def map_cor(a,b):
    """
    Correlation coefficient between two firing rate maps
    
    Arguments:
    a: 2D np.array (map1)
    b: 2D np.array (map2)
    
    Returns:
    Pearson correlation coefficient between a and b
    """
    a = a.flatten()
    b = b.flatten()
    indices = np.logical_and(~np.isnan(a), ~np.isnan(b))
    if np.sum(indices)<2:
        return np.nan
    r,p = pearsonr(a[indices],b[indices])
    return r

In [10]:
fn =myProject.dataPath+"/results/myMaps.pickle" #, dictionary with key per condition
with open(fn, 'rb') as handle:
    myMaps = pickle.load(handle)

In [11]:
myMaps.keys()

dict_keys(['circ80', 'circ80_1', 'circ80_2', 'task', 'task_1', 'task_2', 'light', 'light_arena', 'dark', 'dark_arena', 'light_1', 'light_2', 'dark_1', 'dark_2', 'all_light', 'all_light_1', 'all_light_2', 'all_dark', 'all_dark_1', 'all_dark_2'])

In [12]:
env1 = ["circ80_1","task_1","circ80","all_light","circ80","circ80","all_light_1","all_dark_1"]
env2 = ["circ80_2","task_2","task","all_dark","all_light","all_dark", "all_light_2","all_dark_2"]
condNames = ["oo","tt","ot","ld","ol","od","ll","dd"]
list(zip(env1,env2,condNames))

[('circ80_1', 'circ80_2', 'oo'),
 ('task_1', 'task_2', 'tt'),
 ('circ80', 'task', 'ot'),
 ('all_light', 'all_dark', 'ld'),
 ('circ80', 'all_light', 'ol'),
 ('circ80', 'all_dark', 'od'),
 ('all_light_1', 'all_light_2', 'll'),
 ('all_dark_1', 'all_dark_2', 'dd')]

In [13]:
mapCorDf = pd.DataFrame()
nCells=myMaps["circ80"].shape[0]
for e1,e2,c in zip(env1,env2,condNames):
    print(e1,e2,c)
    correlation  = [ map_cor(myMaps[e1][i],myMaps[e2][i]) for i in range(nCells) ]
    df1 = pd.DataFrame({"condition":c,
                       "r":correlation})
    mapCorDf = pd.concat([mapCorDf,df1])

circ80_1 circ80_2 oo
task_1 task_2 tt
circ80 task ot


/home/maryam/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


all_light all_dark ld
circ80 all_light ol
circ80 all_dark od


/home/maryam/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/maryam/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/maryam/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


all_light_1 all_light_2 ll
all_dark_1 all_dark_2 dd


/home/maryam/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/maryam/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


In [14]:
mapCorDf

,condition,r
0,oo,0.573875
1,oo,0.502913
2,oo,0.854963
3,oo,NaN
4,oo,0.902165
...,...,...
1075,dd,0.155251
1076,dd,0.837723
1077,dd,0.333613
1078,dd,0.823293


### Save the DataFrame


In [15]:
fn=myProject.dataPath+"/results/mapCorDf.csv"
print("Saving to",fn)
mapCorDf.to_csv(fn,index=False)

Saving to /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/results/mapCorDf.csv
